In [3]:
pip install rapidfuzz


  Obtaining dependency information for rapidfuzz from https://files.pythonhosted.org/packages/75/67/62e57896ecbabe363f027d24cc769d55dd49019e576533ec10e492fcd8a2/rapidfuzz-3.13.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from rapidfuzz import fuzz

# Load data
officers = pd.read_csv('/Users/yangyungchyi/Documents/Spring 2025/MIS 462/Officer.csv', dtype=str)
metadata = pd.read_csv('/Users/yangyungchyi/Documents/Spring 2025/MIS 462/0420_4.csv', dtype=str)

# Function to determine if the officer title is relevant
def is_relevant_title(title):
    title = str(title).lower()
    if any(ex in title for ex in ['secr', 'vice', 'assistant', 'past', 'elect', 'trustee', 'officer', 'chairman']):
        return False
    score_president = fuzz.partial_ratio(title, 'president')
    score_ceo = fuzz.partial_ratio(title, 'ceo')
    score_owner = fuzz.partial_ratio(title, 'owner')
    return score_president > 80 or score_ceo > 80 or score_owner > 80

# Filter officers by relevant titles
filtered_officers = officers[officers['OfficerTitle'].apply(is_relevant_title)]

# Merge metadata with filtered officers
merged = pd.merge(
    metadata,
    filtered_officers,
    on='EntityID',
    how='left',
    suffixes=('', '_officer')
)

# Rename officer columns from the merged dataframe
merged = merged.rename(columns={
    'OfficerFirstName_officer': 'OfficerFirstName',
    'OfficerLastName_officer': 'OfficerLastName',
    'OfficerTitle_officer': 'OfficerTitle'
})

# Reorder columns: move officer fields to the end
officer_cols = ['OfficerLastName', 'OfficerFirstName', 'OfficerTitle']
other_cols = [col for col in merged.columns if col not in officer_cols]
merged = merged[other_cols + officer_cols]

columns_to_keep = [
    "EntityID", "Name", "Status", "IncorpDate", "IncorpState", "BuildingName", "Street1", "Street2",
    "City", "State", "BusinessType", "ZIPCODE", "Shape__Area", "Shape__Length", 
    "Full_Address", "Latitude", "Longitude",'OfficerFirstName','OfficerLastName','OfficerTitle'
]

merged = merged[columns_to_keep]

# Save result
merged.to_csv("0420_without_building.csv", index=False)
